In [1]:
import tensorflow as tf

### Tensorflow Graphs

Tensorflow is different from normal programming. With Tensorflow we first
build a computational graph. Then later we can supply values for the nodes
of the graph and run it.

In [2]:
a = tf.constant(1.0)
b = tf.constant(2.0)

c = a*b
print(a)
print(b)
print(c)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("mul:0", shape=(), dtype=float32)


### Session
To run a tensorflow graph we need to create a session. The session allows
us to tell the graph which nodes to compute and returns their values.

In [3]:
sess = tf.Session()
c_value = sess.run(c)
print(c_value)

2.0


### Placeholders
To supply inputs to our graph we need to use placeholder tensors.

These are tensors that have an unspecified value that we supply as input
when running the graph.

In [4]:
a = tf.placeholder(shape=[3], dtype=tf.float32)
b = tf.Variable(tf.ones([3]))

c = a+b
print(a)
print(b)
print(c)

Tensor("Placeholder:0", shape=(3,), dtype=float32)
<tf.Variable 'Variable:0' shape=(3,) dtype=float32_ref>
Tensor("add:0", shape=(3,), dtype=float32)


### Setting placeholder values with a feed dict

When running the graph, the run function lets us specify the value of the
placeholder tensors with a dictionary. It then uses these values when 
computing graph nodes.

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
b_value = sess.run(b)
c_value = sess.run(c,feed_dict={a:[1.0,2.0,3.0]})
print(b_value)
print(c_value)

[1. 1. 1.]
[2. 3. 4.]


### Variable Scope
Tensorflow graphs create a unique string identifier for each node.
This is known as variable scope.
We can group variables together using the scope.

In [6]:
with tf.variable_scope('my_scope'):
    b = tf.Variable(tf.ones([3]), name='b')
    
print(b)

<tf.Variable 'my_scope/b:0' shape=(3,) dtype=float32_ref>


we can retrieve the variable elsewhere in our code by referring to its scope.

In [7]:
c = tf.get_variable('my_scope/c',shape=[3],
                    initializer=tf.ones_initializer())
    
print(c)

<tf.Variable 'my_scope/c:0' shape=(3,) dtype=float32_ref>


In [8]:
with tf.variable_scope('my_scope',reuse=True):
    d = tf.get_variable('c')
    
print(d)

<tf.Variable 'my_scope/c:0' shape=(3,) dtype=float32_ref>


### Optimizing graphs
We can optimize the values of variables in our graph using builtin optimizers

We do this by defining a **loss tensor**.

Tensorflow then can compute the gradients of the loss with
respect to all the nodes in the graph and optimize their values.

In [9]:
x = tf.placeholder(shape=[3],dtype=tf.float32)
y = tf.placeholder(shape=[3],dtype=tf.float32)
weights = tf.get_variable('w',shape=[3],initializer=tf.ones_initializer)

#This is the loss tensor
MSE = tf.reduce_mean(tf.square(weights*x-y))

### Optimizers
Tensorflow provides optimizers.

These optimizers create an optimization tensor that, when run,
computes a step of the optimization algorithm.

In [10]:
learning_rate = 0.1
momentum = 0.9
opt = tf.train.MomentumOptimizer(learning_rate,momentum)
train = opt.minimize(MSE)

In [11]:
x_feed = [1,-1,2]
y_feed = [2,3,4]
sess.run(tf.global_variables_initializer())
for i in range(100):
    sess.run(train,{x:x_feed,y:y_feed})
    
print(sess.run(weights))

[ 1.9966023 -2.9864092  2.00517  ]


In [12]:
input_size = 100
output_size = 10
init = tf.random_normal_initializer(mean=0.0,stddev=1e-2)

x = tf.placeholder(shape=[1,input_size],dtype=tf.float32)
y = tf.placeholder(shape=[1,output_size],dtype=tf.float32)

with tf.variable_scope('layer_1'):
    W = tf.get_variable('W',[input_size,output_size],
                        initializer=init)
    b = tf.get_variable('b',[output_size],
                        initializer=init)
    
    h = tf.matmul(x,W)+b
    a = tf.nn.sigmoid(h)
    
MSE = tf.reduce_mean(tf.square(a-y))